In [1]:
pip install utils

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 23.2.1 -> 23.3.2
[notice] To update, run: C:\Users\Dell\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip


In [2]:
pip install keras-tuner


[notice] A new release of pip is available: 23.2.1 -> 23.3.2
[notice] To update, run: C:\Users\Dell\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip


In [3]:
import os
import random
import shutil

import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Dense, Flatten, Dropout, BatchNormalization
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from kerastuner.tuners import RandomSearch

C:\Users\Dell\AppData\Local\Temp\ipykernel_6032\2465427622.py:11: DeprecationWarning: `import kerastuner` is deprecated, please use `import keras_tuner`.
  from kerastuner.tuners import RandomSearch


In [4]:
# Set the paths to the original dataset and the split dataset
data_dir = 'C:\\Users\\Dell\\OneDrive\\Desktop\\TB\\data'
train_dir = os.path.join(data_dir, "train")
val_dir = os.path.join(data_dir, "val")
test_dir = os.path.join(data_dir, "test")

# Create the train, validation, and test directories if they don't exist
os.makedirs(train_dir, exist_ok=True)
os.makedirs(val_dir, exist_ok=True)
os.makedirs(test_dir, exist_ok=True)

# Set the split ratios
train_ratio = 0.7
val_ratio = 0.15
test_ratio = 0.15

# Loop through each class (Normal and Tuberculosis)
for class_name in ["Normal", "Tuberculosis"]:
    # Create the class directories in the train, validation, and test directories
    os.makedirs(os.path.join(train_dir, class_name), exist_ok=True)
    os.makedirs(os.path.join(val_dir, class_name), exist_ok=True)
    os.makedirs(os.path.join(test_dir, class_name), exist_ok=True)

    # Get the list of images for the current class
    images = os.listdir(os.path.join(data_dir, class_name))
    random.shuffle(images)

    # Calculate the number of images for each split
    num_train = int(len(images) * train_ratio)
    num_val = int(len(images) * val_ratio)
    num_test = len(images) - num_train - num_val

    # Split the images and move them to the respective directories
    train_images = images[:num_train]
    val_images = images[num_train:num_train+num_val]
    test_images = images[num_train+num_val:]

    for image in train_images:
        src = os.path.join(data_dir, class_name, image)
        dst = os.path.join(train_dir, class_name, image)
        shutil.copyfile(src, dst)

    for image in val_images:
        src = os.path.join(data_dir, class_name, image)
        dst = os.path.join(val_dir, class_name, image)
        shutil.copyfile(src, dst)

    for image in test_images:
        src = os.path.join(data_dir, class_name, image)
        dst = os.path.join(test_dir, class_name, image)
        shutil.copyfile(src, dst)


In [5]:
def build_model(hp):
    model = Sequential([
        Conv2D(filters=hp.Int("filters_1", 32, 64, step=32),
               kernel_size=hp.Choice("kernel_size_1", [3, 5]),
               activation="relu",
               input_shape=(150, 150, 3)),
        MaxPooling2D(pool_size=(2, 2)),
        Conv2D(filters=hp.Int("filters_2", 64, 128, step=32),
               kernel_size=hp.Choice("kernel_size_2", [3, 5]),
               activation="relu"),
        MaxPooling2D(pool_size=(2, 2)),
        Conv2D(filters=hp.Int("filters_3", 128, 256, step=32),
               kernel_size=hp.Choice("kernel_size_3", [3, 5]),
               activation="relu"),
        MaxPooling2D(pool_size=(2, 2)),
        Flatten(),
        Dense(units=hp.Int("units", 128, 256, step=32), activation="relu"),
        Dropout(hp.Float("dropout", 0.25, 0.5, step=0.05)),
        Dense(2, activation="softmax")
    ])

    model.compile(optimizer=Adam(learning_rate=hp.Float("learning_rate", 1e-4, 1e-2, sampling="LOG")),
                  loss="sparse_categorical_crossentropy",
                  metrics=["accuracy"])

    return model

data_dir = 'C:\\Users\\Dell\\OneDrive\\Desktop\\TB\\data'
train_dir = os.path.join(data_dir, "train")
val_dir = os.path.join(data_dir, "val")
test_dir = os.path.join(data_dir, "test")

In [6]:
import os

train_dir = 'C:\\Users\\Dell\\OneDrive\\Desktop\\TB\\data\\train'
if not os.path.exists(train_dir):
    os.makedirs(train_dir)

test_dir = 'C:\\Users\\Dell\\OneDrive\\Desktop\\TB\\data\\test'
if not os.path.exists(test_dir):
    os.makedirs(test_dir)

val_dir = 'C:\\Users\\Dell\\OneDrive\\Desktop\\TB\\data\\val'
if not os.path.exists(val_dir):
    os.makedirs(val_dir)



In [7]:
# Data augmentation parameters for the training dataset
train_datagen = ImageDataGenerator(rescale=1./255,        # Rescale pixel values
                                   rotation_range=20,      # Rotate images randomly within a range of -20 to +20 degrees
                                   width_shift_range=0.1,  # Shift images horizontally by 10% of the width
                                   height_shift_range=0.1, # Shift images vertically by 10% of the height
                                   zoom_range=0.1,         # Randomly zoom in or out of the image by a factor of 10%
                                   horizontal_flip=True)   # Randomly flip the image horizontally

val_datagen = ImageDataGenerator(rescale=1./255)
test_datagen = ImageDataGenerator(rescale=1./255)

In [8]:
train_generator = train_datagen.flow_from_directory(train_dir, target_size=(150, 150), batch_size=32, class_mode="binary")
val_generator = val_datagen.flow_from_directory(val_dir, target_size=(150, 150), batch_size=32, class_mode="binary")
test_generator = test_datagen.flow_from_directory(test_dir, target_size=(150, 150), batch_size=32, class_mode="binary")




Found 3822 images belonging to 2 classes.
Found 1159 images belonging to 2 classes.
Found 1174 images belonging to 2 classes.


In [9]:
tuner = RandomSearch(build_model,
                     objective="val_accuracy",
                     max_trials=1,
                     executions_per_trial=1,
                     directory="random_search",
                     project_name="tb_classification")

tuner.search(train_generator,
             epochs=20,
             validation_data=val_generator,
             callbacks=[tf.keras.callbacks.EarlyStopping(monitor="val_loss", patience=3)])


best_model = tuner.get_best_models(num_models=1)[0]
best_model.fit(train_generator, epochs=20, validation_data=val_generator, callbacks=[tf.keras.callbacks.EarlyStopping(monitor="val_loss", patience=3)])

test_loss, test_acc = best_model.evaluate(test_generator)



Trial 1 Complete [00h 29m 54s]
val_accuracy: 0.9361518621444702

Best val_accuracy So Far: 0.9361518621444702
Total elapsed time: 00h 29m 54s

Epoch 1/20
120/120 [==============================] - 329s 3s/step - loss: 0.2781 - accuracy: 0.8579 - val_loss: 0.1307 - val_accuracy: 0.9396
Epoch 2/20
120/120 [==============================] - 142s 1s/step - loss: 0.2711 - accuracy: 0.8791 - val_loss: 0.2432 - val_accuracy: 0.8999
Epoch 3/20
120/120 [==============================] - 156s 1s/step - loss: 0.2528 - accuracy: 0.9014 - val_loss: 0.2029 - val_accuracy: 0.9180
Epoch 4/20
37/37 [==============================] - 17s 449ms/step - loss: 0.1833 - accuracy: 0.9506


In [10]:
print(f"Test loss: {test_loss:.4f}, Test accuracy: {test_acc:.4f}")

Test loss: 0.1833, Test accuracy: 0.9506


In [12]:
# Correct variable and path
best_model.save("C:\\Users\\Dell\\OneDrive\\Desktop\\TB\\TB_model.h5") 


C:\Users\Dell\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\keras\src\engine\training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(
